# Backpropagation - Nätverket som lär sig självt

I förra notebooken byggde vi ett XOR-nätverk med manuellt inställda vikter. Det fungerade, men det är opraktiskt för riktiga problem med tusentals neuroner.

Nu ska vi lära oss **Backpropagation** - algoritmen som låter neurala nätverk hitta sina egna vikter genom att propagera felet bakåt genom nätverket.

## Gradient Descent

När nätverket gissar fel uppstår en **total kostnad** (felet). Denna kostnad är beroende av alla vikter i hela nätverket tillsammans. Ändrar vi en enda vikt litegrann, så kommer den totala kostnaden att förändras. Målet är att justera nätverket så att vi närmar oss en kostnad på 0 (inget fel!).

Kostnaden kan alltså ses som en funktion av vikterna! **Gradient Descent** är strategin vi använder för att hitta den kombination av vikter som ger lägst möjliga kostnad.

Strategin är super-simpel! För varje enskild vikt behöver vi bara räkna ut:
- Åt vilket håll påverkar den den totala kostnaden?
- Hur mycket?

Denna lutning kallas för **gradienten**.

### Intuition: Gå nedför backen

Tänk dig att du står på en kulle i dimma och vill hitta dalen (lägsta kostnaden). Du kan inte se var dalen är, men du kan känna vilken riktning det lutar under dina fötter. Strategin är enkel:

1. Känn efter vilken riktning det lutar (beräkna gradienten)
2. Ta ett steg i den riktningen (uppdatera vikten)
3. Upprepa tills du når botten

### Uppdateringsregeln

Vår uppdateringsregel för varje enskild vikt blir:

```
ny_vikt = gammal_vikt - α × gradienten_för_denna_vikt
```

Här är `α` (alpha) vår **inlärningsfaktor** som bestämmer hur stora steg vi tar.

- **Hög gradient (brant lutning):** Vi har mycket att vinna på att ändra vikten. Ta större steg!
- **Låg gradient (flack lutning):** Vi är nära botten. Ta mindre steg för att finjustera.

Men hur beräknar vi gradienten för varje vikt, speciellt för vikter i gömda lager? Det är här **Backpropagation** kommer in.

## Backpropagation

Backpropagation är en algoritm för att beräkna gradienten för varje vikt. Den gör detta genom att först beräkna ett **"felansvar"** (delta, δ) för varje enskild neuron.

Olika neuroner kommer alltså vara olika ansvariga för eventuella fel i nätverket. Givet ett fel i slutet, behöver vi hitta vilka som var mest bidragande till felet, och korrigera deras vikter mest!

### De fyra stegen

Processen för ett enskilt tränings-exempel sker alltid i fyra steg:

**Steg 1: Framåtpasset (Forward Pass)**
Skicka in data och låt den flöda framåt genom nätverket. **Spara alla outputs** från varje neuron - vi behöver dem senare!

**Steg 2: Delta för Output-lagret**
För varje neuron i sista lagret:
```
δ_output = (facit - gissning) × derivatan_av_sigmoid(gissning)
```
Varför derivatan? Jo! En hög derivata(lutning) för gissningen betyder att en justering av dess vikter har större påverkan på utfallet. Derivatan är dessutom högre när neuronen är "mindre säker" alltså desto närmre ett värde på 0.5 som gissningen närmar sig. Vi kan tolka det som att vi ger ett högre delta till de neuroner som "inte riktigt har bestämt sig" eller "de som påverkas mest av att justeras".

**Steg 3: Delta för Dolda Lager (baklänges)**
För varje neuron i ett dolt lager:
```
δ_dold = (summerat fel från nästa lager) × derivatan_av_sigmoid(output)
```

**Steg 4: Uppdatera Vikter**
```
ny_vikt = gammal_vikt + α × δ × input_som_passerade_vikten
ny_bias = gammal_bias + α × δ
```

### Sigmoid-derivatan

En smart egenskap hos sigmoid-funktionen är att dess derivata är enkel att beräkna:

Om `s = sigmoid(x)`, då är `sigmoid'(x) = s × (1 - s)`

Vi behöver alltså inte beräkna derivatan från scratch - vi kan använda outputen direkt!

In [3]:
# Setup
import random
import math
import numpy as np

random.seed(42)  # För reproducerbarhet

def sigmoid(x):
    if x < -700:
        return 0.0
    if x > 700:
        return 1.0
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(sig_output):
    """Derivatan av sigmoid, givet sigmoid-outputen."""
    return sig_output * (1 - sig_output)

---

## Övning 7: XOR med gömda lager SOM LÄR SIG

I denna övning ska du implementera backpropagation för ett simpelt XOR-nätverk med ett gömt lager.

Nätverket har:
- 2 inputs
- 2 neuroner i det gömda lagret
- 1 output-neuron

**Din uppgift:** Fyll i TODO:erna för att implementera backpropagation.

In [4]:
class Neuron:
    def __init__(self, num_inputs):
        self.weights = [random.uniform(-1, 1) for _ in range(num_inputs)]
        self.bias = random.uniform(-1, 1)

    def predict(self, inputs):
        total = sum(i * w for i, w in zip(inputs, self.weights)) + self.bias
        return sigmoid(total)


class XORNetwork:
    def __init__(self):
        # Skapa neuronerna explicit
        self.hidden1 = Neuron(num_inputs=2)
        self.hidden2 = Neuron(num_inputs=2)
        self.output_neuron = Neuron(num_inputs=2)

    def predict(self, inputs):
        # Framåtmatning
        h1_out = self.hidden1.predict(inputs)
        h2_out = self.hidden2.predict(inputs)
        final_out = self.output_neuron.predict([h1_out, h2_out])
        return final_out

    def train(self, inputs_list, targets, epochs, learning_rate):
        for epoch in range(epochs):
            for x, target in zip(inputs_list, targets):
                # === FORWARD PASS ===
                h1_out = self.hidden1.predict(x)
                h2_out = self.hidden2.predict(x)
                final_out = self.output_neuron.predict([h1_out, h2_out])

                # TODO 1: Beräkna felet i sista lagret
                # error = facit - output
                output_error = target - final_out

                # TODO 2: Beräkna delta för output-neuronen
                # delta = error × sigmoid_derivative(output)
                output_delta = output_error * sigmoid_derivative(final_out)

                # TODO 3: Beräkna felet för varje hidden neuron
                # Error = delta_nästa × vikt_som_kopplar_dem
                h1_error = output_delta * self.output_neuron.weights[0]
                h2_error = output_delta * self.output_neuron.weights[1]

                # TODO 4: Beräkna delta för varje hidden neuron
                # delta = error × sigmoid_derivative(output)
                h1_delta = h1_error * sigmoid_derivative(h1_out)
                h2_delta = h2_error * sigmoid_derivative(h2_out)

                # TODO 5: Uppdatera vikter för output-neuronen
                # vikt += learning_rate × delta × input_till_vikten
                self.output_neuron.weights[0] += learning_rate * output_delta * h1_out
                self.output_neuron.weights[1] += learning_rate * output_delta * h2_out
                self.output_neuron.bias += learning_rate * output_delta

                # TODO 6: Uppdatera vikter för hidden neuronerna
                self.hidden1.weights[0] += learning_rate * h1_delta * x[0]
                self.hidden1.weights[1] += learning_rate * h1_delta * x[1]
                self.hidden1.bias += learning_rate * h1_delta

                self.hidden2.weights[0] += learning_rate * h2_delta * x[0]
                self.hidden2.weights[1] += learning_rate * h2_delta * x[1]
                self.hidden2.bias += learning_rate * h2_delta

In [5]:
# Träna och testa
xor_inputs = [[0, 0], [0, 1], [1, 0], [1, 1]]
xor_targets = [0, 1, 1, 0]

network = XORNetwork()

print("Tränar XOR-nätverket...")
network.train(xor_inputs, xor_targets, epochs=10000, learning_rate=0.5)

print("\n--- Resultat ---")
for x, t in zip(xor_inputs, xor_targets):
    pred = network.predict(x)
    rounded = round(pred)
    status = "✓" if rounded == t else "✗"
    print(f"Input: {x}, Facit: {t}, Prediktion: {pred:.4f} (≈{rounded}) {status}")

Tränar XOR-nätverket...

--- Resultat ---
Input: [0, 0], Facit: 0, Prediktion: 0.0171 (≈0) ✓
Input: [0, 1], Facit: 1, Prediktion: 0.9805 (≈1) ✓
Input: [1, 0], Facit: 1, Prediktion: 0.9841 (≈1) ✓
Input: [1, 1], Facit: 0, Prediktion: 0.0152 (≈0) ✓


**Fantastiskt!** Om du implementerat TODO:erna korrekt ska nätverket nu kunna lösa XOR - något som var omöjligt med en enda perceptron!

---

## Övning 8: Ett generellt neuralt nätverk

Nu ska du bygga ett fullt kapabelt neuralt nätverk som kan ha godtyckligt antal inputs, outputs och gömda lager.

Öppna `neural_network.py` och granska koden. Den innehåller en halvfärdig implementation av `NeuralNetwork`-klassen med backpropagation. Du har några TODO:s där som du borde kunna lösa när du förstått backpropagation.

Kör sedan exemplet nedan för att testa ditt nätverk!

Arkitekturen definieras med `layer_sizes`. Till exempel:
- `[2, 2, 1]` = 2 inputs, 2 dolda neuroner, 1 output (XOR)
- `[64, 30, 10]` = 64 inputs, 30 dolda, 10 outputs (siffror)

In [ ]:
from neural_network import NeuralNetwork

# Testa på XOR
X_train = [[0, 0], [0, 1], [1, 0], [1, 1]]
y_train = [[0], [1], [1], [0]]

nn = NeuralNetwork(layer_sizes=[2, 2, 1])

print("Startar träning av neuralt nätverk för XOR...")
nn.train(X_train, y_train, epochs=10000, learning_rate=0.5)

print("\nTestresultat:")
for inputs, target in zip(X_train, y_train):
    prediction = nn.predict(inputs)
    rounded = round(prediction[0])
    status = "✓" if rounded == target[0] else "✗"
    print(f"Input: {inputs} -> Gissning: {prediction[0]:.4f} (≈{rounded}), Facit: {target[0]} {status}")

---

## Övning 9: Modifiera ett Neuralt Nätverk

**Syfte:** Att förstå hur arkitekturen i ett neuralt nätverk påverkar dess förmåga att lära sig.

### Experiment 1: Flaskhals - Färre Neuroner

Ändra arkitekturen från `[2, 2, 1]` till `[2, 1, 1]`. Du har nu skapat en "flaskhals" med bara en enda neuron i det dolda lagret.

In [ ]:
# Experiment 1: Flaskhals
nn_bottleneck = NeuralNetwork(layer_sizes=[2, 1, 1])

print("Tränar nätverk med FLASKHALS [2, 1, 1]...")
nn_bottleneck.train(X_train, y_train, epochs=10000, learning_rate=0.5)

print("\nResultat med flaskhals:")
correct = 0
for inputs, target in zip(X_train, y_train):
    prediction = nn_bottleneck.predict(inputs)
    rounded = round(prediction[0])
    if rounded == target[0]:
        correct += 1
    status = "✓" if rounded == target[0] else "✗"
    print(f"Input: {inputs} -> {prediction[0]:.4f} (≈{rounded}), Facit: {target[0]} {status}")

print(f"\nNoggrannhet: {correct}/4")

**Fråga:** Kan nätverket fortfarande lösa XOR? Varför/varför inte?

> *Hint: En enda neuron i det gömda lagret kan bara skapa en rak linje. XOR kräver minst två linjer för att separera datan.*

### Experiment 2: Djupare Nätverk

Ändra arkitekturen till `[2, 4, 4, 1]`. Du har nu skapat ett djupare nätverk med två dolda lager.

In [ ]:
# Experiment 2: Djupare nätverk
nn_deep = NeuralNetwork(layer_sizes=[2, 4, 4, 1])

print("Tränar DJUPT nätverk [2, 4, 4, 1]...")
nn_deep.train(X_train, y_train, epochs=10000, learning_rate=0.5)

print("\nResultat med djupt nätverk:")
correct = 0
for inputs, target in zip(X_train, y_train):
    prediction = nn_deep.predict(inputs)
    rounded = round(prediction[0])
    if rounded == target[0]:
        correct += 1
    status = "✓" if rounded == target[0] else "✗"
    print(f"Input: {inputs} -> {prediction[0]:.4f} (≈{rounded}), Facit: {target[0]} {status}")

print(f"\nNoggrannhet: {correct}/4")

**Fråga:** Lär sig det djupare nätverket snabbare eller långsammare? Blir resultatet mer stabilt?

> **Reflektera:** Du har nu agerat som en AI-arkitekt. Hur påverkade antalet neuroner och lager modellens prestanda? Detta är kärnan i **hyperparameter-tuning**: att hitta den bästa arkitekturen och de bästa inställningarna för ett givet problem.

### Experiment 3: Jämför olika arkitekturer

In [ ]:
import matplotlib.pyplot as plt

architectures = [
    [2, 2, 1],
    [2, 3, 1],
    [2, 4, 1],
    [2, 2, 2, 1],
    [2, 4, 4, 1],
]

results = []

for arch in architectures:
    nn = NeuralNetwork(layer_sizes=arch)
    nn.train(X_train, y_train, epochs=5000, learning_rate=0.5)
    
    # Beräkna MSE
    mse = 0
    for x, y in zip(X_train, y_train):
        pred = nn.predict(x)
        mse += (y[0] - pred[0]) ** 2
    mse /= len(X_train)
    
    results.append((str(arch), mse))
    print(f"Arkitektur {arch}: MSE = {mse:.6f}")

---

## Vad har vi lärt oss?

I denna notebook har vi:

1. Förstått **Gradient Descent** - att gå nedför kostnadslandskapet
2. Lärt oss **Backpropagation** - att propagera felet bakåt för att hitta varje vikts bidrag
3. Implementerat backprop för ett **XOR-nätverk** med gömda lager
4. Byggt ett **generellt `NeuralNetwork`** med godtycklig arkitektur
5. Experimenterat med olika arkitekturer och sett hur de påverkar lärandet

## Nästa steg

I nästa notebook använder vi vårt färdiga nätverk för att lösa riktiga problem:
- Handskrivna siffror
- Regression (huspriser)
- Text (Bag of Words)
- Bilder (Autoencoder)

Vi kommer också lära oss om de två stora familjerna av maskininlärning: **Supervised** och **Unsupervised Learning**.